In [10]:
import math
from sympy import *
import numpy as np

Lagrange interpolating polynomial of degree at most $n$:

In [9]:
def L(n,k,xx,lx):
    '''
    n: interpolating polynomical of degree at most n
    k: intermediate degree
    lx: list of initial given values x0, x1, x2, [x0,x1,x2]
    
    '''
    result=1
    for i in range(0,n+1):
        if i!=k:
            result=result*((xx-lx[i])/(lx[k]-lx[i]))
        else:
            result=result
    return result

def P(f,xx,lx,n):
    '''
    f: sympy expression of function f(x)
    lx: list of initial given values x0, x1, x2, [x0,x1,x2]
    n: interpolating polynomical of degree at most n
    
    returns the appoximation of f(xx)
    
    '''
    f = lambdify(x, f)
    px=0
    for k in range(0,n+1):
        px+=(f(lx[k])*L(n,k,xx,lx))
    print("f(",xx,") ≈ ",px, sep="")
    return px

Neville's method:

In [ ]:
def neville(xx, vec_x, vec_f, Q_table = None, i0 = -1, j0 = -1):
    '''
    reference: http://web.math.ucsb.edu/~atzberg/pmwiki_intranet/uploads/Teaching2019Fall104A/Nevilles-Method-Atzberger.pdf
    '''
    n = np.size(vec_x) - 1;  # x0, x1, ..., xn.

    if (Q_table == None):
        Q_table = np.zeros((n + 1, n + 1));

    for i in np.arange(0, n + 1):
        Q_table[i][0] = vec_f[i];
  
    for j in np.arange(1, n + 1):
        for i in np.arange(j, n + 1):
            # compute Q_{i,j}
            Q_table[i][j]  = 0.0;
            Q_table[i][j] += (xx - vec_x[i - j])*Q_table[i][j - 1];
            Q_table[i][j] -= (xx - vec_x[i])*Q_table[i - 1][j - 1];
            Q_table[i][j] /= (vec_x[i] - vec_x[i - j]);
                   
    return Q_table[n][n], Q_table;    

Newton's divided difference method:

In [11]:
def DivDiff(xx, vec_x, vec_f, F_table = None, i0 = -1, j0 = -1):
    n = np.size(vec_x) - 1;  # x0, x1, ..., xn.

    if (F_table == None):
        F_table = np.zeros((n + 1, n + 1));

    for i in np.arange(0, n + 1):
        F_table[i][0] = vec_f[i];
  
    for j in np.arange(1, n + 1):
        for i in np.arange(j, n + 1):
        # compute F_{i,j}
            F_table[i][j]  = 0.0;
            F_table[i][j] += F_table[i][j - 1];
            F_table[i][j] -= F_table[i - 1][j - 1];
            F_table[i][j] /= (vec_x[i] - vec_x[i-j]);
        
    P_x=F_table[0][0]
    P_xx=F_table[0][0] #expression in x
    for i in range(1,n+1):
        product=1
        productx=1
        for j in range(0,i):
            #compute P_n_(x)
            product*=(xx-vec_x[j])
            productx*=(x-vec_x[j])
        P_x+=F_table[i][i]*product
        P_xx+=F_table[i][i]*productx

    return P_x, P_xx, F_table; 

Newton forward-difference method:

In [ ]:
def ForwardDivDiff(xx, vec_x, vec_f, F_table = None, i0 = -1, j0 = -1):
    n = np.size(vec_x) - 1;  # x0, x1, ..., xn.
    h=vec_x[1]-vec_x[0]
    ss=(xx-vec_x[0])/h
    if (F_table == None):
        F_table = np.zeros((n + 1, n + 1));

    for i in np.arange(0, n + 1):
        F_table[i][0] = vec_f[i];
  
    for j in np.arange(1, n + 1):
        for i in np.arange(j, n + 1):
        # compute F_{i,j}
            F_table[i][j]  = 0.0;
            F_table[i][j] += F_table[i][j - 1];
            F_table[i][j] -= F_table[i - 1][j - 1];
            F_table[i][j] /= (vec_x[i] - vec_x[i - j]);
        
    P_x=F_table[0][0]
    P_s=F_table[0][0]
    for k in range(1,n+1):
        product=1
        products=1
        for it in range(0,k):
            #compute \binom{s}{k}*h^k
            product*=ss-it
            products*=s-it
        P_x+=product*pow(h,k)*F_table[k][k]
        P_s+=products*pow(h,k)*F_table[k][k]

    return P_x, P_s, F_table;

Newton backward-difference method:

In [12]:
def BackwardDivDiff(xx, vec_x, vec_f, F_table = None, i0 = -1, j0 = -1):
    n = np.size(vec_x) - 1;  # x0, x1, ..., xn.
    h=vec_x[1]-vec_x[0]
    ss=(xx-vec_x[n])/h
    if (F_table == None):
        F_table = np.zeros((n + 1, n + 1));

    for i in np.arange(0, n + 1):
        F_table[i][0] = vec_f[i];
  
    for j in np.arange(1, n + 1):
        for i in np.arange(j, n + 1):
        # compute F_{i,j}
            F_table[i][j]  = 0.0;
            F_table[i][j] += F_table[i][j - 1];
            F_table[i][j] -= F_table[i - 1][j - 1];
            F_table[i][j] /= (vec_x[i] - vec_x[i - j]);
        
    P_x=F_table[n][0]
    P_s=F_table[n][0]
    for k in range(1,n+1):
        product=1
        products=1
        for it in range(0,k):
            #compute \binom{-s}{k}*h^k
            product*=ss+it
            products*=s+it
        P_x+=product*pow(h,k)*F_table[n][k]
        P_s+=products*pow(h,k)*F_table[n][k]

    return P_x, P_s, F_table;

Stirling's formula:

In [13]:
def Stirling(xx, vec_x, vec_f, F_table = None, i0 = -1, j0 = -1):
    n=np.size(vec_x)-1
    c = int(np.size(vec_x)/2);  # x_-c, x_{-c+1}, ...,x0,..., xc.
    h=vec_x[1]-vec_x[0]
    ss=(xx-vec_x[c])/h
    if (F_table == None):
        F_table = np.zeros((n + 1, n + 1));

    for i in np.arange(0, n + 1):
        F_table[i][0] = vec_f[i];
  
    for j in np.arange(1, n + 1):
        for i in np.arange(j, n + 1):
        # compute F_{i,j}
            F_table[i][j]  = 0.0;
            F_table[i][j] += F_table[i][j - 1];
            F_table[i][j] -= F_table[i - 1][j - 1];
            F_table[i][j] /= (vec_x[i] - vec_x[i - j]);
        
    P_x=F_table[c][0]
    for k in range(1,n+1):
        productOdd=ss
        productEven=1
        for m in range(0,int(k/2)):
            if(k%2==1):
                productOdd*=pow(ss,2)-pow(m+1,2)
            if(k%2==0):
                productEven*=pow(ss,2)-pow(m,2) 
        if(k%2==1):
            P_x+=productOdd*pow(h,k)*(1/2)*(F_table[c+int((k-1)/2)][k]+F_table[c+int((k-1)/2)+1][k])
        if(k%2==0):
            P_x+=productEven*pow(h,k)*(F_table[c+int(k/2)][k])

    return P_x, F_table;

Hermite Interpolation:

In [ ]:
def Hermite(xx, vec_x, vec_f, vec_df, Q_table = None, i0 = -1, j0 = -1):
    n=np.size(vec_x)-1
    if (Q_table == None):
        Q_table = np.zeros((2*n + 2, 2*n + 2));
    z=list(range(2*n+2))
    for i in np.arange(0, n + 1):
        z[2*i]=vec_x[i]
        z[2*i+1]=vec_x[i]
        Q_table[2*i][0] = vec_f[i];
        Q_table[2*i+1][0] = vec_f[i]
        Q_table[2*i+1][1]=vec_df[i]
        if i!=0:
            Q_table[2*i][1]=(Q_table[2*i][0]-Q_table[2*i-1][0])/(z[2*i]-z[2*i-1])
  
    for i in np.arange(2, 2*n + 2):
        for j in np.arange(2, i+1):
        # compute Q_{i,j}
            Q_table[i][j]  = 0.0;
            Q_table[i][j] += Q_table[i][j - 1];
            Q_table[i][j] -= Q_table[i - 1][j - 1];
            Q_table[i][j] /= (z[i] - z[i - j]);
        
    H_x=Q_table[0][0]
    for i in range(1,2*n+2):
        productOdd=xx-vec_x[int((i-1)/2)]
        productEven=1
        for j in range(0,int(i/2)):
            if(i%2==1):
                productOdd*=pow(xx-vec_x[j],2)
            if(i%2==0):
                productEven*=pow(xx-vec_x[j],2)
        if(i%2==1):
            H_x+=Q_table[i][i]*productOdd
        if(i%2==0):
            H_x+=Q_table[i][i]*productEven

    return H_x, Q_table; 

Natural Cubic Spline:

In [ ]:
def NaturalCS(vec_x, vec_f):
    n=np.size(vec_x)-1
    a=[]
    for m in vec_f:
        a.append(m)
    h=[]
    for i in range(0,n):
        h.append(vec_x[i+1]-vec_x[i])
    alpha=[0]*n
    for i in range(1,n):
        alpha[i]+=(3/h[i])*(a[i+1]-a[i])-(3/h[i-1])*(a[i]-a[i-1])
    l=[1]
    u=[0]
    z=[0]
    for i in range(1,n):
        l.append(2*(vec_x[i+1]-vec_x[i-1])-h[i-1]*u[i-1])
        u.append(h[i]/l[i])
        z.append((alpha[i]-h[i-1]*z[i-1])/l[i])
    l.append(1)
    z.append(0)
    c=[0]*(n+1)
    b=[0]*n
    d=[0]*n
    j=n-1
    while j>=0:
        c[j]=z[j]-u[j]*c[j+1]
        b[j]=(a[j+1]-a[j])/h[j]-h[j]*(c[j+1]+2*c[j])/3
        d[j]=(c[j+1]-c[j])/(3*h[j])
        j-=1
    return a,b,c,d
# S_j(x)=a[j]+b[j]*(x-vec_x[j])+c[j]*pow(x-vec_x[j],2)+d[j]*pow(x-vec_x[j],3)

Bezier Curve:

In [14]:
def Bezier(endpts,guidepts):
    n=len(endpts)-1  #P_0, ..., P_n
    a0=[]
    b0=[]
    a1=[]
    b1=[]
    a2=[]
    b2=[]
    a3=[]
    b3=[]
    for i in range(0,n):
        a0.append(endpts[i][0])  #x_i
        b0.append(endpts[i][1])  #y_i
        a1.append(3*(guidepts[2*i][0]-endpts[i][0]))
        b1.append(3*(guidepts[2*i][1]-endpts[i][1]))
        a2.append(3*(endpts[i][0]+guidepts[2*i+1][0]-2*guidepts[2*i][0]))
        b2.append(3*(endpts[i][1]+guidepts[2*i+1][1]-2*guidepts[2*i][1]))
        a3.append(endpts[i+1][0]-endpts[i][0]+3*guidepts[2*i][0]-3*guidepts[2*i+1][0])
        b3.append(endpts[i+1][1]-endpts[i][1]+3*guidepts[2*i][1]-3*guidepts[2*i+1][1])
    C=[]
    for i in range(0,n):
        xt=a0[i]+a1[i]*t+a2[i]*pow(t,2)+a3[i]*pow(t,3)
        yt=b0[i]+b1[i]*t+b2[i]*pow(t,2)+b3[i]*pow(t,3)
        C.append((xt,yt))
    return C  

Forward-difference formula:

In [ ]:
def forwardDF(vec_x,f):
    f=lambdify(x,f)
    h=vec_x[1]-vec_x[0]
    vec_df=[]
    n=np.size(vec_x)-1
    for i in range(0,n):
        print(((f(vec_x[i+1])-f(vec_x[i]))/h))
        vec_df.append((f(vec_x[i+1])-f(vec_x[i]))/h)
    return vec_df

Backward-difference formula:

In [ ]:
def backwardDF(vec_x,f):
    f=lambdify(x,f)
    h=vec_x[0]-vec_x[1]
    vec_df=[]
    n=np.size(vec_x)-1
    for i in range(1,n+1):
        vec_df.append((f(vec_x[i-1])-f(vec_x[i]))/h)
    return vec_df

Three-point endpoint formula:

In [ ]:
def find(x0,vec_x):
    l=np.size(vec_x)
    for i in range(0,l):
        if x0== vec_x[i]:
            return i
    return -1 # cannot find

def threePtEnd(x0,vec_x,vec_f):
    h=vec_x[1]-vec_x[0]
    i=find(x0,vec_x)
    l=np.size(vec_x)
    if i==l-1:
        dfx0=(vec_f[i-2]-4*vec_f[i-1]+3*vec_f[i])/(2*h)
    else:
        dfx0=(-3*vec_f[i]+4*vec_f[i+1]-vec_f[i+2])/(2*h)
    return dfx0

Three-point midpoint formula:

In [ ]:
def threePtMid(x0,vec_x,vec_f):
    h=vec_x[1]-vec_x[0]
    i=find(x0,vec_x)
    dfx0=(vec_f[i+1]-vec_f[i-1])/(2*h)
    return dfx0

Trapezoidal rule:

In [ ]:
def trapezoidal(x0,x1,f): 
    f=lambdify(x,f)
    h=x1-x0
    intf=(f(x0)+f(x1))*h/2
    return intf

Simpson's rule:

In [16]:
def simpson(x0,x2,f): 
    f=lambdify(x,f)
    x1=(x2+x0)/2
    h=x1-x0
    intf=(f(x0)+4*f(x1)+f(x2))*h/3
    return intf

Euler's method:

In [ ]:
def Euler(f,a,b,alpha,h):
    '''
    f: function:
    a,b: endpoints
    alpha: initial condition
    h: step size'''
    f=lambdify((t,y),f)
    tt=a
    ww=alpha
    N=int((b-a)/h)
    ti=[]
    wi=[]
    for i in range(1,N+1):
        ww=ww+h*f(tt,ww)
        tt=a+i*h
        ti.append(tt)
        wi.append(ww)
    return ti,wi